## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("FX-READY").getOrCreate()

# بعد ذلك اقرأ الملف
file_location = "fx_rates_sample.csv"  # أو المسار الفعلي للملف
df = spark.read.csv(file_location, header=True, inferSchema=True)
df.show(5)









+-------------------+----+---+--------+----------+--------+
|          timestamp|base|USD|     SAR|       YER|     EUR|
+-------------------+----+---+--------+----------+--------+
|2025-09-01 09:00:00| USD|1.0|3.750305|249.969495|0.849856|
|2025-09-01 09:01:00| USD|1.0|3.749265|250.046174|0.851548|
|2025-09-01 09:02:00| USD|1.0|3.750015|250.246139|0.848127|
|2025-09-01 09:03:00| USD|1.0|3.750956|250.034432|0.845882|
|2025-09-01 09:04:00| USD|1.0|3.749005| 250.00943|0.844497|
+-------------------+----+---+--------+----------+--------+
only showing top 5 rows


In [2]:
df.printSchema()

root
 |-- timestamp: timestamp (nullable = true)
 |-- base: string (nullable = true)
 |-- USD: double (nullable = true)
 |-- SAR: double (nullable = true)
 |-- YER: double (nullable = true)
 |-- EUR: double (nullable = true)



In [3]:
df.columns

['timestamp', 'base', 'USD', 'SAR', 'YER', 'EUR']

In [4]:
### Handling Categorical Features
from pyspark.ml.feature import StringIndexer

In [5]:
df.show()

+-------------------+----+---+--------+----------+--------+
|          timestamp|base|USD|     SAR|       YER|     EUR|
+-------------------+----+---+--------+----------+--------+
|2025-09-01 09:00:00| USD|1.0|3.750305|249.969495|0.849856|
|2025-09-01 09:01:00| USD|1.0|3.749265|250.046174|0.851548|
|2025-09-01 09:02:00| USD|1.0|3.750015|250.246139|0.848127|
|2025-09-01 09:03:00| USD|1.0|3.750956|250.034432|0.845882|
|2025-09-01 09:04:00| USD|1.0|3.749005| 250.00943|0.844497|
|2025-09-01 09:05:00| USD|1.0|3.747703|250.305721|0.846689|
|2025-09-01 09:06:00| USD|1.0| 3.74783|250.157004|0.847113|
|2025-09-01 09:07:00| USD|1.0|3.747514|249.992554|0.848264|
|2025-09-01 09:08:00| USD|1.0|3.747497|250.033015|0.846554|
|2025-09-01 09:09:00| USD|1.0|3.746644|250.201892|0.844874|
|2025-09-01 09:10:00| USD|1.0|3.747524|250.204177|0.845546|
|2025-09-01 09:11:00| USD|1.0|3.748301|250.469969|0.845634|
|2025-09-01 09:12:00| USD|1.0|3.748368|250.641328|0.846457|
|2025-09-01 09:13:00| USD|1.0|3.749495|2

In [6]:
indexer = StringIndexer(inputCol="base", outputCol="base_indexed")
df_r = indexer.fit(df).transform(df)
df_r.show(5)

+-------------------+----+---+--------+----------+--------+------------+
|          timestamp|base|USD|     SAR|       YER|     EUR|base_indexed|
+-------------------+----+---+--------+----------+--------+------------+
|2025-09-01 09:00:00| USD|1.0|3.750305|249.969495|0.849856|         0.0|
|2025-09-01 09:01:00| USD|1.0|3.749265|250.046174|0.851548|         0.0|
|2025-09-01 09:02:00| USD|1.0|3.750015|250.246139|0.848127|         0.0|
|2025-09-01 09:03:00| USD|1.0|3.750956|250.034432|0.845882|         0.0|
|2025-09-01 09:04:00| USD|1.0|3.749005| 250.00943|0.844497|         0.0|
+-------------------+----+---+--------+----------+--------+------------+
only showing top 5 rows


In [7]:
indexer = StringIndexer(inputCol="base", outputCol="base_indexed")
df_r = indexer.fit(df).transform(df)
df_r.show(5)

+-------------------+----+---+--------+----------+--------+------------+
|          timestamp|base|USD|     SAR|       YER|     EUR|base_indexed|
+-------------------+----+---+--------+----------+--------+------------+
|2025-09-01 09:00:00| USD|1.0|3.750305|249.969495|0.849856|         0.0|
|2025-09-01 09:01:00| USD|1.0|3.749265|250.046174|0.851548|         0.0|
|2025-09-01 09:02:00| USD|1.0|3.750015|250.246139|0.848127|         0.0|
|2025-09-01 09:03:00| USD|1.0|3.750956|250.034432|0.845882|         0.0|
|2025-09-01 09:04:00| USD|1.0|3.749005| 250.00943|0.844497|         0.0|
+-------------------+----+---+--------+----------+--------+------------+
only showing top 5 rows


In [8]:
df_r.columns

['timestamp', 'base', 'USD', 'SAR', 'YER', 'EUR', 'base_indexed']

In [9]:
from pyspark.ml.feature import VectorAssembler
featureassembler = VectorAssembler(
    inputCols=['USD', 'SAR', 'YER', 'EUR'],  # الأعمدة الرقمية الموجودة
    outputCol='features'
)

output = featureassembler.transform(df_r)  # أو df إذا لم تعدّل الاسم
output.select('features').show(5)

+--------------------+
|            features|
+--------------------+
|[1.0,3.750305,249...|
|[1.0,3.749265,250...|
|[1.0,3.750015,250...|
|[1.0,3.750956,250...|
|[1.0,3.749005,250...|
+--------------------+
only showing top 5 rows


In [10]:
output.select('features').show(5)


+--------------------+
|            features|
+--------------------+
|[1.0,3.750305,249...|
|[1.0,3.749265,250...|
|[1.0,3.750015,250...|
|[1.0,3.750956,250...|
|[1.0,3.749005,250...|
+--------------------+
only showing top 5 rows


In [11]:
output.show()

+-------------------+----+---+--------+----------+--------+------------+--------------------+
|          timestamp|base|USD|     SAR|       YER|     EUR|base_indexed|            features|
+-------------------+----+---+--------+----------+--------+------------+--------------------+
|2025-09-01 09:00:00| USD|1.0|3.750305|249.969495|0.849856|         0.0|[1.0,3.750305,249...|
|2025-09-01 09:01:00| USD|1.0|3.749265|250.046174|0.851548|         0.0|[1.0,3.749265,250...|
|2025-09-01 09:02:00| USD|1.0|3.750015|250.246139|0.848127|         0.0|[1.0,3.750015,250...|
|2025-09-01 09:03:00| USD|1.0|3.750956|250.034432|0.845882|         0.0|[1.0,3.750956,250...|
|2025-09-01 09:04:00| USD|1.0|3.749005| 250.00943|0.844497|         0.0|[1.0,3.749005,250...|
|2025-09-01 09:05:00| USD|1.0|3.747703|250.305721|0.846689|         0.0|[1.0,3.747703,250...|
|2025-09-01 09:06:00| USD|1.0| 3.74783|250.157004|0.847113|         0.0|[1.0,3.74783,250....|
|2025-09-01 09:07:00| USD|1.0|3.747514|249.992554|0.848264| 

In [12]:
finalized_data = output.select('features', 'YER')

In [13]:
finalized_data.show()

+--------------------+----------+
|            features|       YER|
+--------------------+----------+
|[1.0,3.750305,249...|249.969495|
|[1.0,3.749265,250...|250.046174|
|[1.0,3.750015,250...|250.246139|
|[1.0,3.750956,250...|250.034432|
|[1.0,3.749005,250...| 250.00943|
|[1.0,3.747703,250...|250.305721|
|[1.0,3.74783,250....|250.157004|
|[1.0,3.747514,249...|249.992554|
|[1.0,3.747497,250...|250.033015|
|[1.0,3.746644,250...|250.201892|
|[1.0,3.747524,250...|250.204177|
|[1.0,3.748301,250...|250.469969|
|[1.0,3.748368,250...|250.641328|
|[1.0,3.749495,250...|250.809692|
|[1.0,3.749962,250...|250.920515|
|[1.0,3.749103,251...|251.386046|
|[1.0,3.749472,251...|251.345014|
|[1.0,3.748513,250...|250.944309|
|[1.0,3.749391,251...| 251.26516|
|[1.0,3.749341,251...| 251.17362|
+--------------------+----------+
only showing top 20 rows


In [14]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

# 1) تجهيز الـ features
assembler = VectorAssembler(
    inputCols=['USD','SAR','EUR','base_indexed'],
    outputCol='features'
)
output = assembler.transform(df_r)
finalized_data = output.select('features', 'YER')

# 2) تقسيم البيانات
train_data, test_data = finalized_data.randomSplit([0.75, 0.25])

# 3) تدريب النموذج
regressor = LinearRegression(featuresCol='features', labelCol='YER')
model = regressor.fit(train_data)

# 4) التقييم (هنا نستخدم model وليس regressor)
pred_results = model.evaluate(test_data)

print("RMSE:", pred_results.rootMeanSquaredError)
print("R2:", pred_results.r2)

# عرض التنبؤات
pred_results.predictions.show(5)


RMSE: 1.5117301064719657
R2: 0.09328775126408262
+--------------------+----------+------------------+
|            features|       YER|        prediction|
+--------------------+----------+------------------+
|[1.0,3.736651,0.8...| 247.51477|247.46401773241524|
|[1.0,3.737278,0.8...|247.163454|247.55448265840516|
|[1.0,3.737362,0.8...| 247.19316| 247.5698746795548|
|[1.0,3.737703,0.8...|249.728291|247.61894684520502|
|[1.0,3.737876,0.8...|249.855786| 247.6352252050914|
+--------------------+----------+------------------+
only showing top 5 rows


In [15]:
## Final comparison
pred_results.predictions.show()

+--------------------+----------+------------------+
|            features|       YER|        prediction|
+--------------------+----------+------------------+
|[1.0,3.736651,0.8...| 247.51477|247.46401773241524|
|[1.0,3.737278,0.8...|247.163454|247.55448265840516|
|[1.0,3.737362,0.8...| 247.19316| 247.5698746795548|
|[1.0,3.737703,0.8...|249.728291|247.61894684520502|
|[1.0,3.737876,0.8...|249.855786| 247.6352252050914|
|[1.0,3.738307,0.8...| 247.11908|247.67530943356434|
|[1.0,3.738344,0.8...|246.743588|247.67015275902327|
|[1.0,3.738405,0.8...|246.759906|247.66785653832187|
|[1.0,3.738839,0.8...|246.071196|247.74222965020624|
|[1.0,3.739009,0.8...|246.930851|247.76921686756185|
|[1.0,3.739009,0.8...|247.528082|247.75867485675133|
|[1.0,3.739056,0.8...|248.841364|247.73059467450452|
|[1.0,3.739105,0.8...|247.173018| 247.7624382659908|
|[1.0,3.739234,0.8...|249.375766| 247.7707840422504|
|[1.0,3.739247,0.8...|247.880139|247.78575281287942|
|[1.0,3.739404,0.8...|247.012371|247.796078989

In [16]:
### PErformance Metrics
pred_results.r2,pred_results.meanAbsoluteError,pred_results.meanSquaredError

(0.09328775126408262, 1.2273812162726638, 2.285327914813741)